## Importing Necessary Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd


## Loading the Documents

In [2]:
import pandas as pd

chunk_size = 10000
chunks = pd.read_csv("C:\\Users\\srile\\Downloads\\aws_review_sofware_dataset.csv", sep=',', chunksize=chunk_size)

# Get the first chunk and access its columns
df = next(chunks)
print(df.columns)

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [4]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

## Importing sent Tokenize from NLTK

In [5]:
from nltk.tokenize import sent_tokenize

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
for i in range(499):
    l1= sent_tokenize(df.loc[i,"reviewText"])
    df.at[i,"sentences"]=l1

In [8]:

# pywsd library is a tool for resolving word ambiguity in NLP
!pip install pywsd

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# Importing averaged_perceptron_tagger finding POS_tagging
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 7.682044506072998 secs.


In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:

for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [14]:
df["words_sentences"] = "default"

In [15]:
# It is commonly used for tasks like memoization, partial function application, and modifying function properties.
import functools

# Iterate through the DataFrame
for k in range(df.shape[0]):
    words = df.loc[k, "words"]
    # Check if words is empty and handle accordingly
    if words:
        # Join the words into a single string
        df.loc[k, "words_sentences"] = functools.reduce(lambda a, b: str(a) + " " + str(b), words)
    else:
        # If the list is empty, set the column to an empty string or some default value
        df.loc[k, "words_sentences"] = ""

## Importing TF-IDF Vectorizer

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [17]:
import sklearn.feature_extraction.text
dir(sklearn.feature_extraction.text)


['BaseEstimator',
 'CountVectorizer',
 'ENGLISH_STOP_WORDS',
 'FLOAT_DTYPES',
 'FeatureHasher',
 'HasMethods',
 'HashingVectorizer',
 'Integral',
 'Interval',
 'Mapping',
 'NotFittedError',
 'OneToOneFeatureMixin',
 'RealNotInt',
 'StrOptions',
 'TfidfTransformer',
 'TfidfVectorizer',
 'TransformerMixin',
 '_IS_32BIT',
 '_VectorizerMixin',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_analyze',
 '_check_stop_list',
 '_document_frequency',
 '_fit_context',
 '_make_int_array',
 '_preprocess',
 'array',
 'check_array',
 'check_is_fitted',
 'defaultdict',
 'itemgetter',
 'normalize',
 'np',
 'partial',
 're',
 'sp',
 'strip_accents_ascii',
 'strip_accents_unicode',
 'strip_tags',
 'unicodedata',
 'warnings']

In [18]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

In [19]:
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())
df_y=df["verified"]

## Importing LabelEncoder

In [20]:

from sklearn.preprocessing import LabelEncoder

In [21]:
le=LabelEncoder()

In [22]:
df_y_1=pd.DataFrame(df_y)

In [23]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [24]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(dense, df_y_enc)

# Accuracy
accuracy_rf = rf.score(dense, df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 64.15%


In [26]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(dense, df_y_enc)

# Compute accuracy
accuracy_nb = nb.score(dense, df_y_enc)
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes Accuracy: 63.64%


In [27]:
from sklearn.ensemble import GradientBoostingClassifier

In [28]:
GBC=GradientBoostingClassifier(n_estimators=1000)

In [29]:
gb_c = GBC.fit(dense, df_y_enc)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
gbc_score=GBC.score(dense, df_y_enc)
print(f"gbc_score: {gbc_score* 100:.2f}%")

gbc_score: 64.15%
